In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import color
from time import time
from scipy import signal
import imageio

In [ ]:
img = plt.imread('/content/drive/MyDrive/COL783/1024px-Broadway_tower_edit.jpg')

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img)
plt.title('Original Image')

In [ ]:
#computes the energy (gradient magnitude) of a given image
def energyfunction(img): 
  img1 = np.copy(img)
  img1 = color.rgb2gray(img1)
  #finding the gradient in x and y dir
  sobelx = cv2.Sobel(img1,cv2.CV_64F,1,0,ksize=5)
  sobely = cv2.Sobel(img1,cv2.CV_64F,0,1,ksize=5)
  #energy function
  energy = abs(sobelx) + abs(sobely)
  return energy

#overlay the seam on the original image in red color
def plotseam(img,paths,col):
  img3 = np.copy(img)
  for i in range(img.shape[0]):
    indx = np.int(img.shape[0]-1-i)
    img3[indx,col] = [255,0,0]
    col = np.int(col + paths[indx,col] - 1)
    
  plt.figure(figsize=(10,10))
  plt.title('Seam')
  plt.imshow(img3)
  plt.show()
  return img3


#find the seam using the energy of the image
#function returns cumulative energy(dp) and
#paths array to backtrack the seam
def computeSeam(energy):
  dp = np.copy(energy)
  paths = np.zeros(dp.shape)
  for i in range(1,dp.shape[0]):
    for j in range(dp.shape[1]):
      if (j>=1 and j+1<dp.shape[1]):
        dp[i,j] = dp[i,j] + np.min([dp[i-1,j-1],dp[i-1,j],dp[i-1,j+1]])
        paths[i,j] = np.int(np.argmin([dp[i-1,j-1],dp[i-1,j],dp[i-1,j+1]]))
      elif (j+1>=dp.shape[1]):
        dp[i,j] = dp[i,j] + np.min([dp[i-1,j-1],dp[i-1,j]])
        paths[i,j] = np.int(np.argmin([dp[i-1,j-1],dp[i-1,j]]))
      elif (j<1):
        dp[i,j] = dp[i,j] + np.min([dp[i-1,j],dp[i-1,j+1]])
        paths[i,j] = np.int(1 + np.argmin([dp[i-1,j],dp[i-1,j+1]]))
        
  return dp,paths

In [ ]:
#remove the seam from the image, energy and mask
def removeSeam(img,energy,mask):
  dp,paths = computeSeam(energy)
  #find min seam
  col = np.int(np.argmin(dp[dp.shape[0]-1,:]))
  #visulaizing the seam
  img3 = plotseam(img,paths,col)
  #img2 will store the image after the removal of the seam
  img2 = np.zeros((img.shape[0],img.shape[1] - 1,3))

  #remove the seam from mask as well as the energy function
  mask1 = np.zeros((mask.shape[0],mask.shape[1]-1))
  energy1 = np.zeros((energy.shape[0],energy.shape[1]-1))

  for i in range(img2.shape[0]):
    indx = np.int(img2.shape[0]-1-i)
   #columns before the seam as it is cols after get shifted by one
    if (col+1<img.shape[1]):
      img2[indx][0:col] = img[indx][0:col]
      img2[indx][col:] = img[indx][1+col:]
      mask1[indx][:col] = mask[indx][:col]
      mask1[indx][col:] = mask[indx][1+col:]
      energy1[indx][0:col] = energy[indx][:col]
      energy1[indx][col:] = energy[indx][1+col:]
    else:
      img2[indx] = img[indx,:col]
      mask1[indx] = mask[indx,:col]
      energy1[indx] = energy[indx,:col]

    col = np.int(col + paths[indx,col] - 1)

  print("Image size after seam removal: ",img2.shape[0],img2.shape[1])
  #img2: contains image with object removed, img3 contains plotted seam
  return img2.astype('uint8'), mask1.astype('int'),energy1.astype('int'),img3.astype('uint8')

In [ ]:
def removeObject(img,x1,y1,x2,y2):
  img_list = []
  #plot the object to be removed
  plt.figure(figsize=(10,10))
  plt.imshow(img[y1:y2,x1:x2])
  plt.title('Object to be removed')
  plt.show()
  #bounding box
  img1 = np.copy(img)
  img1 = cv2.rectangle(img1,(x1,y1),(x2,y2),(255,0,0))
  print(type(img1))
  try:
    img1 = cv2.UMat.get(img1)
  except:
    pass

  plt.figure(figsize=(10,10))
  plt.imshow(img1)
  plt.title('Bounding box')
  plt.show()
  #create a mask of the object
  mask = np.ones((img.shape[0],img.shape[1]))
  mask[y1:y2,x1:x2] = 0

  #compute the energy of the image
  energy = energyfunction(img)

  #to make all the seams pass through the object
  #make the values negative inside the box
  energy1 = energy*mask - 1000*(1-mask)*energy

  #dimensions of the box
  nx = np.abs(x2 - x1)
  ny = np.abs(y2 - y1)

  img1 = np.copy(img)
  mask1 = np.copy(mask)
  energy2 = np.copy(energy1)

  #remove vertical seam
  if (ny > 0.5*nx):
    for i in range(nx):
      print('Iteratio:',i)
      img1,mask1,energy2,img3 = removeSeam(img1,energy2,mask1)
      img_list.append(img1)
      img_list.append(img3)
  #remove horizontal seam
  else:
    #rotate the image,energy and mask
    img1 = np.rot90(img1)
    energy2 = np.rot90(energy2)
    mask1 = np.rot90(mask1)
    for i in range(ny):
      print('Iteratio:',i)
      img1,mask1,energy2,img3 = removeSeam(img1,energy2,mask1)
    img1 = np.rot90(img1)
    img1 = np.rot90(img1) 
    img1 = np.rot90(img1)
    img3 = np.rot90(img3)
    img3 = np.rot90(img3) 
    img3 = np.rot90(img3)

    img_list.append(img1)
    img_list.append(img3)  


  plt.figure(figsize=(10,10))
  plt.imshow(img1)
  plt.title('Result')
  return img1,img_list

In [ ]:
#insert seam
def plotseam_insert(img,cols):
  img1 = np.copy(img)
  for i in range(img.shape[0]):
    indx = img.shape[0] -1 - i
    img1[indx,cols[i]] = [255,0,0]
  #plt.imshow(img1)
  #plt.show()
  return img1

def plotseam1(img,cols):
  img1 = np.copy(img)
  for i in range(img.shape[0]):
    indx = img.shape[0] -1 - i
    img1[indx,1+cols[i]] = [255,0,0]
  #plt.imshow(img1)
  #plt.show()
  return img1

def findSeam(energy):
  dp = np.copy(energy)
  paths = np.zeros(dp.shape)
  for i in range(1,dp.shape[0]):
    for j in range(dp.shape[1]):
      if (j-1>=0 and j+1<dp.shape[1]):
        dp[i,j] = dp[i,j] + np.min([dp[i-1,j-1],dp[i-1,j],dp[i-1,j+1]])
        paths[i,j] = np.int(np.argmin([dp[i-1,j-1],dp[i-1,j],dp[i-1,j+1]]))
      elif (j+1>=dp.shape[1]):
        dp[i,j] = dp[i,j] + np.min([dp[i-1,j-1],dp[i-1,j]])
        paths[i,j] = np.int(np.argmin([dp[i-1,j-1],dp[i-1,j]]))
      elif (j-1<0):
        dp[i,j] = dp[i,j] + np.min([dp[i-1,j],dp[i-1,j+1]])
        paths[i,j] = np.int(1 + np.argmin([dp[i-1,j],dp[i-1,j+1]]))

  #now we have cum energy, now we have to find the min seam
  cols = []
  col = np.argmin(dp[dp.shape[0]-1,:])
  cols.append(col)
  for i in range(dp.shape[0]-1):
    indx = dp.shape[0] - 1 - i
    col = np.int(col + paths[indx,col] - 1)
    cols.append(col)

  return cols

def deleteseam(img,cols):
  img1 = np.zeros((img.shape[0],img.shape[1]-1,3))
  for i in range(img1.shape[0]):
    indx = img1.shape[0] - 1 - i
    col = cols[i]
    img1[indx,:col] = img[indx,:col]
    if (1+col < img.shape[1]):
      img1[indx,col:] = img[indx,1+col:]
  
  return img1.astype('uint8')

def insertseam(img,cols):
  img1 = np.zeros((img.shape[0],img.shape[1]+1,3))
  img = img.astype('float32')
  img1 = img1.astype('float32')
  for i in range(img.shape[0]):
    indx = img.shape[0]-i-1
    col = cols[i]
    img1[indx,:1+col] = img[indx,:1+col]

    if (col-1>=0 and col+1<img.shape[1]):
      img1[indx,1+col,0] = np.int((img[indx,col-1][0] + img[indx,col+1,0])/2)
      img1[indx,1+col,1] = np.int((img[indx,col-1][1] + img[indx,col+1,1])/2)
      img1[indx,1+col,2] = np.int((img[indx,col-1][2] + img[indx,col+1,2])/2)
    elif (col-1 < 0):
      img1[indx,1+col] = img[indx,col+1]
    elif (col+1 >= img.shape[1]):
      img1[indx,1+col] = img[indx,col-1]
    
    if (1+col<img.shape[1]):
      img1[indx,col+2:] = img[indx,col+1:]
    
  return img1.astype('uint8')


def enlargevert(img,dy):
  img_list_video = []
  img1 = np.copy(img)
  #add dy vertical seams
  seam_list = []
  #first find dy seams
  #store the seams in seam_list
  for i in range(dy):
    energy = energyfunction(img1)
    cols = findSeam(energy)
    seam_list.append(cols)
    img1 = deleteseam(img1,cols)  

  #now insert seam
  img2 = np.copy(img)
  for i in range(dy):
    img2 = insertseam(img2,seam_list[i])
    img_list_video.append(img2)
    img3 = plotseam_insert(img2,seam_list[i])
    img_list_video.append(img3)

    '''
    print(img2.shape)
    plt.imshow(img2)
    plt.show()
    plt.imshow(img3)
    plt.show()
    '''

  img4 = np.copy(img2)
  for i in range(dy):
    img4 = plotseam1(img4,seam_list[i])
  

  return img2,img_list_video,img4

def enlarge(img,dx,dy):
  img_list = []
  #add vertical seams
  img1 = np.copy(img)
  img2,images,seamimg = enlargevert(img1,dy)
  img_list.append(images)

  plt.figure(figsize=(10,10))
  plt.title('Image with added seams')
  plt.imshow(seamimg)
  plt.show()

  #add horizontal seams
  img3 = np.copy(img2)
  img3 = np.rot90(img3)
  img4,images,seamimg = enlargevert(img3,dx)

  seamimg = np.rot90(seamimg)
  seamimg = np.rot90(seamimg)
  seamimg = np.rot90(seamimg)

  plt.figure(figsize=(10,10))
  plt.title('Image with added seams')
  plt.imshow(seamimg)
  plt.show()

  img4 = np.rot90(img4)
  img4 = np.rot90(img4) 
  img4 = np.rot90(img4)

  img_list.append(images)

  return img4.astype('uint8'),img_list



In [ ]:
#experiment1
img1 = np.copy(img)
plt.title('Original Imgae')
plt.imshow(img1)

In [ ]:
#bounding box
x1,y1 = [80,520]
x2,y2 = [110,580]
img2 = np.copy(img1)
img2 = cv2.rectangle(img2,(x1,y1),(x2,y2),(255,0,0))
plt.figure(figsize=(10,10))
plt.imshow(img2)
plt.title('Bounding box')

In [ ]:
#remove the object
img1 = np.copy(img)
img2,img_list1 = removeObject(img1,x1,y1,x2,y2)

In [ ]:
for i in range(len(img_list1)):
  plt.imshow(img_list1[i])
  plt.show()

In [ ]:
#inserting seams to make the object the same size
#insert seams
dx = x2 - x1
dy = y2 - y1
print(dx,dy)
if (dy > dx):
  img3,img_list2 = enlarge(img2,0,dx)
else:
  img3,img_list2 = enlarge(img2,dy,0)

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.imshow(img)
plt.title('Original Image')
plt.subplot(1,2,2)
plt.imshow(img3)
plt.title('Image after removing object')
print('The shape of original image: ',img.shape)
print('The shape of the result: ',img3.shape)

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.imshow(img)
plt.title('Original Image')
plt.subplot(1,2,2)
plt.imshow(img3)
plt.title('Image after removing object')
print('The shape of original image: ',img.shape)
print('The shape of the result: ',img3.shape)

In [ ]:
print('Image list 0')
image_list = []
for i in range(len(img_list1[0])):
  image_list
  plt.imshow(img_list2[0][i])
  plt.show()

print("Image list 1")
for i in range(len(img_list2[0])):
  plt.imshow(img_list2[1][i])
  plt.show()
print('Image list 1 part 2')
for i in range(len(img_list2[1])):
  plt.imshow(img_list2[1][i])
  plt.show()

In [ ]:
#create gif
list1 = img_list1
list2 = img_list2[0]
#list3 = img_list2[1]
list1.extend(list2)
#list1.extend(list3)
imageio.mimsave('objrem1.gif',list1)

In [ ]:
#experiment 2
img6 = plt.imread('/content/drive/MyDrive/COL783/image6.jpg')
plt.imshow(img6)
plt.title('Original Image')

In [ ]:
#bounding box
x1,y1 = [344,232]
x2,y2 = [410,285]
img1 = np.copy(img6)
img1 = cv2.rectangle(img1,(x1,y1),(x2,y2),(255,0,0))
plt.figure(figsize=(10,10))
plt.imshow(img1)
plt.title('Bounding box')

In [ ]:
#remove object
img1 = np.copy(img6)
img2,img2_list1 = removeObject(img1,x1,y1,x2,y2)

In [ ]:
#insert seams
dx = x2 - x1
dy = y2 - y1
print(dx,dy)
if (dy > 0.5*dx):
  img3,img2_list2 = enlarge(img2,0,dx)
else:
  img3,img2_list2 = enlarge(img2,dy,0)

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.imshow(img6)
plt.title('Original Image')
plt.subplot(1,2,2)
plt.imshow(img3)
plt.title('Image after removing object')
print('The shape of original image: ',img6.shape)
print('The shape of the result: ',img3.shape)

In [ ]:
#create gif
list1 = img2_list1
list2 = img2_list2[0]
#list3 = img_list2[1]
list1.extend(list2)
#list1.extend(list3)
imageio.mimsave('objrem2.gif',list1)